# Administration - preamble
If you are seeing this section, then this notebook has been improperly rendered - and will be littered with code and other cells that are supposed to be hidden from general readers.

In [32]:
import os
import sys
import shutil

print(" I: Executing preamble code...", file=sys.stderr)

 I: Executing preamble code...


In [33]:
# --- Preamble code START --- #

# ...just reusing code from a past project of mine, so expect
# inconsistencies in use of language.

from IPython.display import display, Markdown, Latex
import matplotlib.pyplot as plt
import numpy as np
import math

from numpy.fft import fft, ifft
from scipy.signal import find_peaks
from typing import Optional


# - Definitions - #

# Set the notebook name
docname = "text_submissions"
nbname = f"{docname}.ipynb"
pdfname = f"{docname}.pdf"

cwd = os.path.realpath(".")
assert len(cwd) > 0 and cwd != "/"

# Initialize temporary figures directory
figsdir = "tmpfigs"
figsdir_full = f"{cwd}/{figsdir}"

if os.path.exists(figsdir_full):
    shutil.rmtree(figsdir_full)

os.mkdir(figsdir)

print(f"  > Current directory: {cwd}", file=sys.stderr)
print(f"  > Target notebook: {nbname}", file=sys.stderr)
print(f"  > Output file: {pdfname}", file=sys.stderr)
print(f"  > Figures directory: {figsdir_full}", file=sys.stderr)

# Define a counter variable for the figures
FIG_CTR = 1


# - Functions - #

# Clear the current page
def clearpage() -> None:
    display(Latex(r"\clearpage"))


# Display a generic image figure
def disp_imgfig(img: str, desc: str, preamble: str = "", figname: str = "Figure") -> None:
    global FIG_CTR

    # Make sure the file even exists
    if not os.path.exists(img):
        raise ValueError(f"Provided image file '{img}' does not exist.")

    # Create caption
    caption = f"**{figname} {FIG_CTR}:** {desc}"

    # Create Markdown sequence
    md = f"|![{desc}]({img})|\n|:--:|\n|{caption}|"

    if preamble:
        md = preamble + "\n\n" + md

    # Display the sequence
    display(Markdown(md))

    # Increment the figure counter
    FIG_CTR += 1


# Display two generic image figures
def disp_imgfig2(img1: str, img2: str, desc1: str, desc2: str, preamble: str = "") -> None:
    global FIG_CTR

    # Make sure the files even exist
    if not os.path.exists(img1):
        raise ValueError(f"Provided image file '{img1}' does not exist.")

    if not os.path.exists(img2):
        raise ValueError(f"Provided image file '{img2}' does not exist.")

    # Create caption
    caption1 = f"**Figur {FIG_CTR}:** {desc1}"
    caption2 = f"**Figur {FIG_CTR+1}:** {desc2}"

    caption = caption1 + "\\\n" + caption2
    
    # Create Markdown sequence
    md = f"|![{desc1}]({img1})|![{desc2}]({img2})|\n|:--:|:--:|\n|{caption1}|{caption2}|"

    if preamble:
        md = preamble + "\n\n" + md

    # Display the sequence
    display(Markdown(md))

    # Increment the figure counter
    FIG_CTR += 2


# Display a Pyplot figure
def disp_pltfig(fig: plt.Figure, desc: str, preamble: str = "") -> None:    
    global FIG_CTR
    
    # Save the figure as an image file, using the current
    # figure counter to discriminate between figures
    figfile = f"{figsdir}/tmpfig_{FIG_CTR}.png"

    # Save the figure and close it, so that
    # the notebook isn't littered with
    # orphaned figures
    fig.savefig(figfile, bbox_inches="tight", dpi=400)
    plt.close(fig)

    disp_imgfig(figfile, desc, preamble)


# Display two Pyplot figures
def disp_pltfig2(fig1: plt.Figure, fig2: plt.Figure, desc1: str, desc2: str, preamble: str = "") -> None:    
    global FIG_CTR
    
    # Save the figure as an image file, using the current
    # figure counter to discriminate between figures
    figfile_1 = f"{figsdir}/tmpfig_{FIG_CTR}.png"
    figfile_2 = f"{figsdir}/tmpfig_{FIG_CTR+1}.png"

    # Save the figure and close it, so that
    # the notebook isn't littered with
    # orphaned figures
    fig1.savefig(figfile_1, bbox_inches="tight", dpi=400)
    fig2.savefig(figfile_2, bbox_inches="tight", dpi=400)
    plt.close(fig1)
    plt.close(fig2)

    disp_imgfig2(figfile_1, figfile_2, desc1, desc2, preamble)


# Find the `x` coordinates that satisfy `y1[...] == y2`
def where_is(
    x: np.ndarray, 
    y1: np.ndarray, 
    y2: float, 
    err: float = 1.0e-3
) -> list[float]:
    assert len(x) == len(y1)

    a = []

    for u, v in zip(x, y1):
        if math.isclose(v, y2, rel_tol=err):
            a.append(u)

    return np.array(a)


# Define the normalized FFT function
# ...which is really just a shorthand for
# "do everything we would otherwise have
# to write down every time"
# - I find the function signature incredibly funny
# for some reason...
# - The times in `t` should be equidistant
# - the coefficient it returns are complex, because...
def norm_fft(
    t: np.ndarray, 
    x: np.ndarray,
    thr: Optional[float] = None,
) -> (np.ndarray, np.ndarray, np.ndarray):
    # Calculate the sampling rate
    dt = t[1] - t[0]
    f_s = 1 / dt

    # Perform FFT on the input
    X = fft(x)            # FFt of `x`
    N = len(X)            # Number of discrete frequencies
    
    assert N != 0
    
    n = np.arange(N)      # Bin number
    bins = n / N * f_s    # Frequency bins
    X *= 2/N              # Normalize `X` to obtain coefficients
                          # - accounting for symmetry
    
    # - exploit the symmetry of FFT
    bins = bins[:N//2]
    X = X[:N//2]

    # Find peaks
    X_mag = np.abs(X)     # Magnitude of `X`
    peaks, _ = find_peaks(X_mag, threshold=thr)    # Bins with peaks

    return bins, X, peaks


# Return an array of sinusoids with common time and different parameters
def sinusoid(t: np.ndarray, f: np.ndarray, b: np.ndarray) -> np.ndarray:
    # Copy the arrays, which are otherwise mutable
    f_v, t_v, b_v = np.copy(f), np.copy(t), np.copy(b)

    # Turn the arrays into vectors with appropriate shapes
    f_v.shape = (len(f),1)
    t_v.shape = (1,len(t))
    b_v.shape = (1,len(b))

    # Calculate the argument angles, then calculate the result
    phi = 2 * np.pi * (f_v * t_v)
    res = np.sum(b_v.T * np.sin(phi), axis=0)
    return res

# --- Preamble code END --- #

  > Current directory: /home/johnc/Projects/IN3160_mandatory_03
  > Target notebook: text_submissions.ipynb
  > Output file: text_submissions.pdf
  > Figures directory: /home/johnc/Projects/IN3160_mandatory_03/tmpfigs


In [34]:
print(" I: Finished executing preamble code", file=sys.stderr)

 I: Finished executing preamble code


# Prelude
The code examples are taken from the VHDL files in the [`src/`](src/) folder.

# Task 3a
*When does the output data signal change, and what is the cause of this delay?*

In [35]:
disp_imgfig("task_3a_waves.png", "Waves generated by the test bench for task 3a")

|![Waves generated by the test bench for task 3a](task_3a_waves.png)|
|:--:|
|**Figure 1:** Waves generated by the test bench for task 3a|

We can see in the figure above that the delay between *feeding bits into `indata`* and *seeing the same bits out of `outdata`* is exactly 2 clock cycles, excluding the clock cycle where the input bits were introduced to the device-under-test (the *write cycle*). 

```vhdl
-- (original commentary omitted for formatting purposes)
a  <= indata;
v1 := A;
b  <= v1;
v2 := B;
c  <= v2;
```
A variable "remembers" the state of the signal from the exact point the signal was assigned to the variable, though this isn't unique to variables; one can achieve the same effect through the following:
```vhdl
a <= indata;
b <= a;
c <= b;
```
What is common between these approaches, is that they are contained in a clocked process, where signals are "sampled", and then "updated" concurrently for each clock cycle -- and not sequentially. In practice, this means that information "traverses" exactly one signal for each cycle.

Knowing that `a <= indata` and `outdata <= c`, we can confidently assert that the delay is caused by exactly two critical sample-update directives, and that the propagation delay from `indata` and `outdata` is exactly two clock cycles -- excluding the write cycle.

# Task 3b
*Modify `delay.vhd` so that all of the variables are replaced by signals. Modify `tb_delay.py` so that the chip is only in reset from the time 100 ns to 200 ns, and the input data should now change from `00000000` at 0 ns (i.e. start) to `11110000` at time 300 ns and to `00001111` at time 400 ns.*

In [36]:
disp_imgfig("task_3b_waves.png", "Waves generated by the test bench for task 3b")

|![Waves generated by the test bench for task 3b](task_3b_waves.png)|
|:--:|
|**Figure 2:** Waves generated by the test bench for task 3b|

## Question 1
*When does the output data signal change now?*

We can see in figure 2 that changes to `indata` are reflected in `outdata` 4 clock cycles after the write cycle. 

## Question 2
*How and why did changing variables to signals change the timing?*

We have four critical sample-update directives, as opposed to two:
```vhdl
-- (original commentary omitted for formatting purposes)
a  <= indata;
x1 <= a;
b  <= x1;
x2 <= b;
c  <= x2;
```
As we are now using purely signals, we are forced to follow VHDL's sample-update rule for clocked processes, which necessarily leads to delays.

In [37]:
clearpage()

<IPython.core.display.Latex object>

## Question 3
*Why is the output data equal to `UUUUUUUU` at time 50 ns?*

This is because the output signal, as well as the other signals in the chain, remains *unresolved* until the reset line `rst_n` is asserted (here: pulled low):
```vhdl
-- (original commentary omitted for formatting purposes)
architecture rtl of delay_modified is 
    signal a, b, c: std_ulogic_vector(7 downto 0);
    signal x1, x2: std_ulogic_vector(7 downto 0); 
begin  
    process (rst_n, mclk) is    
    begin
        -- if the reset line isn't held at cycle 0, then
        -- all signals *will* be unresolved and undefined
        if (rst_n = '0') then       
            a <= (others => '0');
            x1 <= (others => '0');
            b <= (others => '0');
            x2 <= (others => '0');
            c <= (others => '0');
        elsif rising_edge(mclk) then            
            a  <= indata;
            x1 <= a;
            b  <= x1;
            x2 <= b;
            c  <= x2;
        end if;
    end process;

    outdata  <= c;
end rtl;
```
This necessarily leads to `outdata` also being unresolved until `rst_n` is assertd.

In [38]:
clearpage()

<IPython.core.display.Latex object>

# Task 3c
*Simulate the attached code in `variables_vs_signals.vhd` and `tb_variables_vs_signals.py`.*

## Question 1
*Why is `outdata(7 downto 6)` always equal to `outdata(3 downto 2)`?*

This is because, in a clocked process, only the last assignment to a particular signal is *authoritative*, and is therefore the only one that takes effect. 

We have the following RTL process:
```vhdl
-- (original commentary omitted for formatting purposes)
var1 := indata;
var2 := indata;
sig1 <= var1;
sig2 <= var2;

outdata(1 downto 0) <= var2 & var1;
outdata(3 downto 2) <= sig2 & sig1;

var1 := not var1;
var2 := not var2;

sig1 <= not var1;
sig2 <= not indata;

outdata(5 downto 4) <= var2 & var1;
outdata(7 downto 6) <= sig2 & sig1;
```
We can see that the last assignments to `sig1` and `sig2`, preceded by variable assignments, are
```vhdl
var1 := not var1;
var2 := not var2;

sig1 <= not var1;
sig2 <= not indata;
```
which are authoritative, and resolve to
```vhdl
sig1 <= var1
sig2 <= not indata
```

While this isn't a universal fact, multiple combinational mappings can be placed arbitrarily within the same process, and still "see" the exact same signals -- as assigned by the authoritative assignment. This means that `outdata(7 downto 6)` and `outdata(3 downto 2)` both "see" `sig2 & sig1`, regardless of their placement.

In [39]:
clearpage()

<IPython.core.display.Latex object>

## Question 2
*Why is `outdata(5 downto 4)` different from `outdata(1 downto 0)`?*

This is because, unlike signals -- which tend to be location-agnostic, variables are sensitive to location within the process.

At the risk of regurgitating the RTL process in **question 1**, we have the relevant directives
```vhdl
var1 := indata;
var2 := indata;
outdata(1 downto 0) <= var2 & var1;

var1 := not var1;
var2 := not var2;
outdata(5 downto 4) <= var2 & var1;
```
Although signal assignments in clocked processes are generally concurrent, variables (when assigned to signals) break that concurrency. Instead, the variable states, and therefore the generated signals, must be evaluated sequentially -- just like in software, although the evaluation process is almost always simplified to concurrent primitives whenever possible, so as to not introduce timing and control hazards.

## Question 3
*Draw a diagram showing the architecture. That is: draw the wires and inverters, and fill in where signals and variables occur.*

In [40]:
disp_imgfig("task_3c_drawing.png", "Electrical diagram of the component's architecture")

|![Electrical diagram of the component's architecture](task_3c_drawing.png)|
|:--:|
|**Figure 3:** Electrical diagram of the component's architecture|

In [41]:
clearpage()

<IPython.core.display.Latex object>

# Task 3d
*Remove the signals `sig1` and `sig2` are from the sensitivity list in `variables_vs_signals.vhd`. Why do `outdata(7 downto 6)` and `outdata(3 downto 2)` attain different values compared to ones in task 3c?*

By removing `sig1` and `sig2` from the sensitivity list, they are no longer required to be resolved, and the signals that depend on them don't need to be synchronized with the rest of the process.

In [42]:
disp_imgfig("task_3c_waves.png", "Waves generated by the test bench for task 3c")
disp_imgfig("task_3d_waves.png", "Waves generated by the test bench for task 3d")

|![Waves generated by the test bench for task 3c](task_3c_waves.png)|
|:--:|
|**Figure 4:** Waves generated by the test bench for task 3c|

|![Waves generated by the test bench for task 3d](task_3d_waves.png)|
|:--:|
|**Figure 5:** Waves generated by the test bench for task 3d|

As we can see in figure 4, the directives
```vhdl
outdata(7 downto 6) <= sig2 & sig1
outdata(3 downto 2) <= sig2 & sig1
```
take effect at the same edges where `sig1` and `sig2` change values. This is not the case for `outdata(7 downto 6)` and `outdata(3 downto 2)` as depicted in figure 5, which appear to be 1 rising edge behind.

# Administration - rendering
If you are seeing this section, then this notebook has been improperly rendered - and will be littered with code and other cells that are supposed to be hidden from general readers.

The code below renders this notebook into a PDF file. As the notebook itself has a compliant name, the PDF file will also be rendered with a compliant name.

In [48]:
# The question is: can a cell hide itself
print(f" I: Exporting notebook as PDF... (output file: {pdfname})", file=sys.stderr)

_ = os.system(f'\
jupyter nbconvert \
--to pdf \
--TagRemovePreprocessor.remove_cell_tags="hide_cell" \
--TagRemovePreprocessor.remove_input_tags="hide_input" \
"{nbname}"')

# --- Nothing that is hidden, can be shown beyond this point --- #

 I: Exporting notebook as PDF... (output file: text_submissions.pdf)
[NbConvertApp] Converting notebook text_submissions.ipynb to pdf
[NbConvertApp] Writing 33921 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 217697 bytes to text_submissions.pdf
